In [1]:
import os
import sys
import urllib
from functools import partial
from tkinter import *
import tkinter.filedialog as tkfd
import requests
from PIL import Image, ImageTk

import pathlib


In [2]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from glob import glob
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import glob
from tensorflow import keras

In [3]:
class Fullscreen_Window(Tk):

    def __init__(self):
        self.tk = Tk()
        self.tk.geometry("1000x500")
        self.tk.title('Image Classification')
        self.top = Frame(self.tk)
        self.top.pack(side=TOP)
        self.top2 = Frame(self.tk)
        self.top2.pack(side=TOP)
        self.bottom_left = Frame(self.tk)
        self.bottom_left.pack(side=LEFT, fill=BOTH, expand=True)
        self.bottom_right = Frame(self.tk)
        self.bottom_right.pack(side=RIGHT, fill=BOTH, expand=True)
        self.state = False
        self.tk.bind("f", self.toggle_fullscreen)
        self.tk.bind("e", self.end_fullscreen)

    def toggle_fullscreen(self, event=None):
        self.state = not self.state  # Just toggling the boolean
        self.tk.attributes("-fullscreen", self.state)
        return "break"

    def end_fullscreen(self, event=None):
        self.state = False
        self.tk.attributes("-fullscreen", False)
        return "break"

In [4]:
def show_folder_img():
    global img_in_folder


    print("show_folder_img")
    print("entry에서 받은 주소>>", entry.get())
    new_dir = os.chdir((entry.get()))
    img_in_folder = glob.glob('*.*')
    batch_size = 32
    img_height = 160
    img_width = 160
    channel_n = 3
    
    print("len(img_in_folder)", len(img_in_folder))
    result_dic = sort_img(img_in_folder)

    button_url= []
    i = 1
    for j in img_in_folder:
        button_url.append('b_url'+str(i))
        i+=1
    print(button_url)
    for i, button in enumerate(button_url):
        button = Button(w, text='이미지'+str(i+1), command=partial(showimg,i, result_dic))
        button.pack(pady=10 ,in_=w.top2, side=LEFT)

    # text = Text(w, width=35, height=15)
    # scrollbar = Scrollbar(w)
    # scrollbar.config(command=text.yview)
    # text.config(yscrollcommand=scrollbar.set)
    # scrollbar.pack(in_=w.bottom, side=RIGHT, fill=Y)
    # text.pack(in_=w.bottom, side=LEFT, fill=BOTH, expand=True)


In [5]:
def set_img():
    global entered_img
    my_img = Label(w,
          text='내 이미지',
          font=('comic sans', 10)
          )
    my_img.pack(in_=w.bottom_left, side=TOP)
    entered_img = Image.open(entry.get())
    user_img = ImageTk.PhotoImage(entered_img)
    user_label = Label(w, image=user_img)
    user_label.image = user_img
    user_label.pack(in_=w.bottom_left, side=LEFT)

In [6]:
def clear_img():
    global architecture_label
    global art_and_culture_label
    global food_and_drinks_label
    global travel_and_adventure_label
    global percent_label
    try:
        architecture_label.destroy()
    except Exception as e:
        print("destroy", str(e))
    try:
        art_and_culture_label.destroy()
    except Exception as e:
        print("destroy", str(e))
    try:
        food_and_drinks_label.destroy()
    except Exception as e:
        print("destroy", str(e))
    try:
        travel_and_adventure_label.destroy()
    except Exception as e:
        print("destroy", str(e))
    try:
        percent_label.destroy()
    except Exception as e:
        print(str(e))

In [7]:
# 읽어온 이미지 목록을 스케일링, 리사이징하는 함수 선언
def make_file(data_height, data_width, channel_n, data_list_len, img_in_folder): # 높이, 너비, 컬러, 리스트 길이
    # 4차원으로 만들기 위한 4차원 리스트 필요
    print("make_file img_in_folder>>", img_in_folder)
    images=np.zeros((data_list_len, data_height, data_width, channel_n))
    # 이미지 리스트에서 하나씩 꺼내서 라벨링, 스케일링, 리사이징
    for i, image in enumerate(img_in_folder):     
        # 읽어서 스케일링, 리사이징 
        image=cv2.imread(image) /255 #읽기, 스케일링
        image=cv2.resize(image, (data_height, data_width)) #리사이징
        # 4차원 리스트에 하나씩 append
        images[i, :, :, :]=image
    return images

In [8]:
def sort_img(img_in_folder):
    global architecture_label
    global art_and_culture_label
    global food_and_drinks_label
    global travel_and_adventure_label
    global result_dic
    global percent_dic
    model=load_model('/Users/syshin/Desktop/Himedia/travel_imgclassifier_models/mobile_net_model.h5')
    # 크기를 동일하게 맞추어 변경해주자
    data_height=160
    data_width=160
    channel_n=3 # 컬러 색상으로 설정
    images=make_file(data_height, data_width, channel_n, len(img_in_folder), img_in_folder)


    # 스케일링, 리사이징된 이미지를 모델에 predict
    predicted_label=model.predict(images)
    # 첫번째 이미지의 각 분류 확률
    # predicted_label[0]
    # 이미지가 architecture 카테고리일 확률이 가장 큼
    # np.argmax(predicted_label[0])
    # 분류한 이미지를 각 폴더에 옮기기 위한 라이브러리
    import shutil

    file_source = entry.get()+"/"
    try:
        os.makedirs(file_source+"classified")
        os.makedirs(file_source+"classified/architecture")
        os.makedirs(file_source+"classified/art and culture")
        os.makedirs(file_source+"classified/food and drinks")
        os.makedirs(file_source+"classified/travel and adventure")
    except Exception as e:
        print(str(e))
    file_destination1 = entry.get()+"/classified/architecture/"
    file_destination2 = entry.get()+"/classified/art and culture/"
    file_destination3 = entry.get()+"/classified/food and drinks/"
    file_destination4 = entry.get()+"/classified/travel and adventure/"
    print("img_in_folder>>>>>>>>", img_in_folder)
    print("predicted_label>>", predicted_label)
    result_dic = {}
    percent_dic = {}
    # 가장 높은 확률을 가진 분류 폴더로 이동
    
    
    class_names = ['architecture', 'art and culture', 'food and drinks', 'travel and adventure']
    for i, one in enumerate(img_in_folder):
        img=cv2.imread(one)
        img=cv2.resize(img,(160,160))
        img_array=keras.preprocessing.image.img_to_array(img)
        img_array=tf.expand_dims(img_array,0)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        print(
            i,one,"most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(predictions[0])], 100 * np.max(predictions[0]))
        )
        print(predictions)
        if np.argmax(predictions[0])==0:
            shutil.copyfile(file_source + one, file_destination1 + one)
            result_dic[i]='architecture'
            percent_dic[i]=predictions[0]

        elif np.argmax(predictions[0])==1:
            shutil.copyfile(file_source + one, file_destination2 + one)
            result_dic[i]='art_and_culture'
            percent_dic[i]=predictions[0]

        elif np.argmax(predictions[0])==2:
            shutil.copyfile(file_source + one, file_destination3 + one)
            result_dic[i]='food_and_drinks'
            percent_dic[i]=predictions[0]
           
        else:
            shutil.copyfile(file_source + one, file_destination4 + one)
            result_dic[i]='travel_and_adventure'
            percent_dic[i]=predictions[0]
    return result_dic

In [9]:
def set_start():
    global architecture_label
    global art_and_culture_label
    global food_and_drinks_label
    global travel_and_adventure_label
    global result_dic
    result_label = Label(w,
          text='분류 결과',
          font=('comic sans', 10)
          )
    try:
        result_label.pack(in_=w.bottom_right, side=TOP)
        (x,y)=(120,120)
        architecture_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/architecture.png').resize((x,y))
        art_and_culture_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/art_and_culture.png').resize((x,y))
        food_and_drinks_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/food_and_drinks.png').resize((x,y))
        travel_and_adventure_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/travel_and_adventure.png').resize((x,y))
        
        
        architecture_folder = ImageTk.PhotoImage(architecture_folder_img)
        art_and_culture_folder = ImageTk.PhotoImage(art_and_culture_folder_img)
        food_and_drinks_folder = ImageTk.PhotoImage(food_and_drinks_folder_img)
        travel_and_adventure_folder = ImageTk.PhotoImage(travel_and_adventure_folder_img)


        architecture_label = Label(w, image=architecture_folder)
        art_and_culture_label = Label(w, image=art_and_culture_folder)
        food_and_drinks_label = Label(w, image=food_and_drinks_folder)
        travel_and_adventure_label = Label(w, image=travel_and_adventure_folder)
        
        
        
        architecture_label.image = architecture_folder
        architecture_label.pack(in_=w.bottom_right, side=LEFT)
        
        art_and_culture_label.image = art_and_culture_folder
        art_and_culture_label.pack(in_=w.bottom_right, side=LEFT)
        
        food_and_drinks_label.image = food_and_drinks_folder
        food_and_drinks_label.pack(in_=w.bottom_right, side=LEFT)
        travel_and_adventure_label.image = travel_and_adventure_folder
        travel_and_adventure_label.pack(in_=w.bottom_right, side=LEFT)
    except Exception as e:
        print(str(e))


In [10]:
def showimg(number, result_dic):
    global my_img
    print("showimg: 받은 number=", number)
    global entered_img
    global img_label
    global percent_dic
    global architecture_label
    global art_and_culture_label
    global food_and_drinks_label
    global travel_and_adventure_label
    global percent_label
    clear_img()
    try:
        my_img.destroy()
        img_label.destroy()
    except Exception as e:
        print(str(e))
    my_img = Label(w,
          text='폴더내 이미지',
          font=('comic sans', 10)
          )
    my_img.pack(in_=w.bottom_left, side=TOP)
    
    ## entry에 입력된 폴더 안의 이미지들 가져오기

    entered_img = Image.open(img_in_folder[number]).resize((400,250))
    result_img = ImageTk.PhotoImage(entered_img)
    img_label = Label(w)
    img_label.configure(image=result_img)
    img_label.image = result_img
    img_label.pack(in_=w.bottom_left, side=LEFT)
    (x,y)=(120,120)
    
    if result_dic[number]=='architecture':
        architecture_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/architecture.png').resize((x,y))
        architecture_folder = ImageTk.PhotoImage(architecture_folder_img)
        architecture_label = Label(w, image=architecture_folder)
        architecture_label.image = architecture_folder
        architecture_label.pack(in_=w.bottom_right, side=LEFT) 
        
    elif result_dic[number]=='art_and_culture':
        art_and_culture_label_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/art_and_culture.png').resize((x,y))
        art_and_culture_label_folder = ImageTk.PhotoImage(art_and_culture_label_folder_img)
        art_and_culture_label = Label(w, image=art_and_culture_label_folder)
        art_and_culture_label.image = art_and_culture_label_folder
        art_and_culture_label.pack(in_=w.bottom_right, side=LEFT)
    elif result_dic[number]=='food_and_drinks':
        food_and_drinks_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/food_and_drinks.png').resize((x,y))
        food_and_drinks_folder = ImageTk.PhotoImage(food_and_drinks_folder_img)
        food_and_drinks_label = Label(w, image=food_and_drinks_folder)
        food_and_drinks_label.image = food_and_drinks_folder
        food_and_drinks_label.pack(in_=w.bottom_right, side=LEFT)
    else:
        food_and_drinks_folder_img = Image.open('/Users/syshin/Desktop/Himedia/Himedia_project/image_classification/travel_and_adventure.png').resize((x,y))
        food_and_drinks_folder = ImageTk.PhotoImage(food_and_drinks_folder_img)
        food_and_drinks_label = Label(w, image=food_and_drinks_folder)
        food_and_drinks_label.image = food_and_drinks_folder
        food_and_drinks_label.pack(in_=w.bottom_right, side=LEFT)
        
    percent = round(np.max(percent_dic[number])*100,3)

    percent_label = Label(w,
           text='accuracy:' + str(percent) + "%",
           font=('comic sans', 10)
           )
    percent_label.pack(in_=w.bottom_right, side=LEFT)


In [11]:
def find_img():
    path = tkfd.askopenfilename(initialdir="/", title="Select file", filetypes=(("jpeg files", "*.jpg"), ("all files", "*.*"), ))
    entry.insert('0', path)
def find_folder():
    path = tkfd.askdirectory(initialdir="/",title='Please select a directory')
    entry.insert('0', path)
def delete_images():
    global img_in_folder
    for file in img_in_folder:
        os.remove(file)

In [ ]:
result_dic = {}
user_tag=[]
w = Fullscreen_Window()
entry_guide = Label(w,
           text='사진 선택: ',
           font=('comic sans', 10)
           )
entry_guide.pack(in_=w.top, side=LEFT)  # id라는 라벨을 w에 넣어줌
entry = Entry(w)
entry.pack(in_=w.top, side=LEFT)

s = Button(w, text='폴더 찾아보기', command=find_folder)
s.pack(in_=w.top, side=LEFT)
b = Button(w, text='폴더 입력', command=lambda : [show_folder_img(), set_start()])
b.pack(in_=w.top, side=LEFT)
b = Button(w, text='기존 파일 지우기', command=delete_images)
b.pack(in_=w.top, side=LEFT)

w.tk.mainloop()

show_folder_img
entry에서 받은 주소>> /Users/syshin/Desktop/Himedia/Himedia_project/dongsu_images
len(img_in_folder) 22
Metal device set to: Apple M1 Pro


2022-06-17 12:02:11.449036: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-17 12:02:11.449165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


make_file img_in_folder>> ['KakaoTalk_Photo_2022-06-16-17-02-33 004.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-38 012.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-31 001.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-37 010.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-43 022.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-39 013.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 016.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-39 014.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-32 002.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-34 005.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-32 003.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-35 008.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 017.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-35 007.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-41 018.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-41 019.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-34 006.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 015.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-43 021.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-38 011.jpeg', 'KakaoTalk_Ph

2022-06-17 12:02:16.243138: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-17 12:02:16.480732: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 517ms/step
img_in_folder>>>>>>>> ['KakaoTalk_Photo_2022-06-16-17-02-33 004.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-38 012.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-31 001.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-37 010.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-43 022.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-39 013.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 016.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-39 014.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-32 002.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-34 005.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-32 003.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-35 008.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 017.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-35 007.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-41 018.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-41 019.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-34 006.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-40 015.jpeg', 'KakaoTalk_Photo_2022-06-16-17-02-43 021.jpeg', 'KakaoTalk_P